In [1]:
from google.cloud import bigquery

client = bigquery.Client(project="prod-organize-arizon-4e1c0a83")

#big frames lets you write SQL queries then transform output data with python
query = """
SELECT
c.pctnum
, c.county
, b.congressionaldistrict
, b.statehousedistrict
, CASE
    WHEN a.age < 18 THEN "under 18"
    WHEN a.age BETWEEN 18 AND 24 THEN "18-24"
    WHEN a.age BETWEEN 25 AND 34 THEN "25-34"
    WHEN a.age BETWEEN 35 AND 44 THEN "35-44"
    WHEN a.age BETWEEN 45 AND 54 THEN "45-54"
    WHEN a.age BETWEEN 55 AND 64 THEN "55-64"
    WHEN a.age >= 65 THEN "65+"
    ELSE NULL
  END AS Age_Buckets
, COUNT(a.dwid) AS DWIDS
FROM `proj-tmc-mem-mvp.catalist_enhanced.enh_catalist__ntl_current` AS a
LEFT JOIN `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS b
  ON a.dwid = b.dwid
INNER JOIN `prod-organize-arizon-4e1c0a83.rich_christina_proj.catalist_pctnum_crosswalk_native` AS c
  ON b.uniqueprecinctcode = c.uniqueprecinctcode
WHERE b.state = "AZ"
GROUP BY 1, 2, 3, 4, 5
ORDER BY 1
"""

# Runs query and returns pandas DataFrame
results_df = client.query(query).to_dataframe()

In [ ]:
results_df.head()

In [ ]:
pivot_age_df = results_df.pivot_table(
    index=['pctnum', 'county', 'congressionaldistrict', 'statehousedistrict'],
    columns='Age_Buckets',
    values='DWIDS',
    fill_value=0
).astype(int).reset_index()

pivot_age_df.head()

In [ ]:
# Reorder columns
age_order = [
    'under 18',
    '18-24',
    '25-34',
    '35-44',
    '45-54',
    '55-64',
    '65+'
]
index_cols = ['pctnum', 'county', 'congressionaldistrict', 'statehousedistrict']
pivot_df = pivot_age_df[index_cols + age_order]

# Sanitize column names for BigQuery
pivot_df.columns = (
    pivot_df.columns
    .str.replace(r'[^\w]+', '_', regex=True)
    .str.strip('_')
    .str.lower()
)



pivot_df.head()

In [2]:
from pandas_gbq import to_gbq

project_id = "prod-organize-arizon-4e1c0a83"
dataset_table = "rich_christina_proj.age_counts_pivot"  # dataset.table

# Upload DataFrame to BigQuery
to_gbq(
    pivot_df,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists='replace'  # or 'append'
)

NameError: name 'pivot_df' is not defined